In [1]:
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/Demand Forecasting.csv')
df = df[['A+','A-','B+','B-','AB+','AB-','O+','O-']]
df.head()

,A+,A-,B+,B-,AB+,AB-,O+,O-
0,538,399,675,616,675,529,309,644
1,244,177,124,337,331,210,507,321
2,153,95,601,200,104,479,42,546
3,220,120,40,399,608,589,645,130
4,504,387,65,307,410,406,577,366


In [3]:
def create_time_series(
                        df, 
                        selective_windows = 30,
                        prediction_windows = 1
                        ):
    X, Y = [], []
    for i in range(len(df)):
        end_ix = i + selective_windows
        out_end_ix = end_ix + prediction_windows
        if out_end_ix > len(df):
            break
        seq_x = df[i:end_ix]
        seq_y = df[end_ix:out_end_ix]
        X.append(seq_x)
        Y.append(seq_y)
        
    return np.array(X).squeeze(), np.array(Y).squeeze()

In [4]:
X, Y = create_time_series(df)
print(X.shape, Y.shape)

(370, 30, 8) (370, 8)


In [5]:
def convNet():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv1D(64, 3, activation='relu', input_shape=(X.shape[1], X.shape[2])))
    model.add(tf.keras.layers.MaxPooling1D(2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(50, activation='relu'))
    model.add(tf.keras.layers.Dense(Y.shape[1]))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

model = convNet()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 28, 64)            1600      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 14, 64)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 896)               0         
                                                                 
 dense (Dense)               (None, 50)                44850     
                                                                 
 dense_1 (Dense)             (None, 8)                 408       
                                                                 
Total params: 46,858
Trainable params: 46,858
Non-trainable params: 0
____________________________________________________

In [6]:
model.fit(
        X, Y, 
        epochs=1000, 
        verbose=1
        )

Epoch 1/1000
12/12 [==============================] - 3s 6ms/step - loss: 70019.0625 - mae: 212.9813
Epoch 2/1000
12/12 [==============================] - 0s 4ms/step - loss: 41378.1484 - mae: 170.7184
Epoch 3/1000
12/12 [==============================] - 0s 3ms/step - loss: 38418.8008 - mae: 167.2576
Epoch 4/1000
12/12 [==============================] - 0s 3ms/step - loss: 37037.9688 - mae: 164.3089
Epoch 5/1000
12/12 [==============================] - 0s 3ms/step - loss: 35369.3906 - mae: 160.7926
Epoch 6/1000
12/12 [==============================] - 0s 3ms/step - loss: 34097.2656 - mae: 157.5329
Epoch 7/1000
12/12 [==============================] - 0s 3ms/step - loss: 32895.1211 - mae: 154.8216
Epoch 8/1000
12/12 [==============================] - 0s 4ms/step - loss: 32052.3379 - mae: 152.2807
Epoch 9/1000
12/12 [==============================] - 0s 3ms/step - loss: 30941.1562 - mae: 149.6423
Epoch 10/1000
12/12 [==============================] - 0s 4ms/step - loss: 29941.6992 - mae

In [7]:
model.save('weights/demand/model.h5')